In [1]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc

PROJECT_NAME = "Textile_SY14"
TAR_PATH     = '/etc/data/ecoinvent-3.10-cutoff-bw25.tar.gz'

# If the project already exists, just switch to it,
# otherwise restore from archive and then switch.
if PROJECT_NAME in bd.projects:
    bd.projects.set_current(PROJECT_NAME)
else:
    bi.restore_project_directory(
        fp=TAR_PATH,
        project_name=PROJECT_NAME,
    )
    bd.projects.set_current(PROJECT_NAME)


In [2]:
bd.databases

Databases dictionary with 2 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff

In [3]:
db = bd.Database("ecoinvent-3.10-cutoff")

In [4]:
def get_unique_node(name, location, unit, reference_product, database="ecoinvent-3.10-cutoff"):
    matches = [
        act for act in bd.Database(database)
        if act.get('name') == name
        and act.get('location') == location
        and act.get('unit') == unit
        and act.get('reference product') == reference_product
    ]
    
    if len(matches) == 0:
        raise ValueError(f"No match found for: {name} - {location} - {unit} - {reference_product}")
    elif len(matches) == 1:
        return matches[0]
    elif len(matches) > 1:
        raise ValueError(f"Multiple matches found for: {name} - {location} - {unit} - {reference_product}")

In [11]:
fiber_activities = {}

fiber_activities["acrylic"] = get_unique_node("acrylic acid production", "RER", "kilogram", "acrylic acid")
fiber_activities["cotton"] = get_unique_node("fibre production, cotton, ginning", "RoW", "kilogram", "fibre, cotton")
fiber_activities["polyester"] = get_unique_node("polyester fibre production, finished", "RoW", "kilogram", "fibre, polyester")
fiber_activities["viscose"] = get_unique_node("fibre production, viscose", "GLO", "kilogram", "fibre, viscose")
fiber_activities["silk"] = get_unique_node("market for fibre, silk, short", "GLO", "kilogram", "fibre, silk, short")
fiber_activities["nylon"] = get_unique_node("nylon 6-6 production", "RoW", "kilogram", "nylon 6-6")
fiber_activities["wool"] = get_unique_node("sheep production, for wool", "US", "kilogram", "sheep fleece in the grease")
fiber_activities


{'acrylic': 'acrylic acid production' (kilogram, RER, None),
 'cotton': 'fibre production, cotton, ginning' (kilogram, RoW, None),
 'polyester': 'polyester fibre production, finished' (kilogram, RoW, None),
 'viscose': 'fibre production, viscose' (kilogram, GLO, None),
 'silk': 'market for fibre, silk, short' (kilogram, GLO, None),
 'nylon': 'nylon 6-6 production' (kilogram, RoW, None),
 'wool': 'sheep production, for wool' (kilogram, US, None)}

In [13]:
import numpy as np

# Example: list of activities (products) and LCIA method keys
products = list(fiber_activities.values())
methods = [
    ('IPCC 2021 no LT', 'climate change: fossil no LT', 'global temperature change potential (GTP100) no LT'),
    ('IPCC 2021 no LT', 'climate change: land use no LT', 'global temperature change potential (GTP100) no LT'),
]

results = np.zeros((len(products), len(methods)))

def get_lcia_scores(products, methods, results):
    # Initialize LCA with first product and method
    lca = bc.LCA({products[0]: 1}, methods[0])
    lca.lci()
    lca.lcia()

    # Cache characterization matrices
    method_matrices = [lca.characterization_matrix.copy()]
    for method in methods[1:]:
        lca.switch_method(method)
        method_matrices.append(lca.characterization_matrix.copy())

    # Loop through products and compute scores for each method
    for i, product in enumerate(products):
        try:
            lca.lci({product.id: 1})
            for j, cf_matrix in enumerate(method_matrices):
                results[i, j] = (cf_matrix @ lca.inventory).sum()
        except Exception as e:
            print(f"Error in {product['name']}: {e}")
            results[i, :] = np.nan

    return results

# Run the batch LCA
get_lcia_scores(products, methods, results)

# Display results
for i, product in enumerate(products):
    print(f"{product['name']}:")
    for j, method in enumerate(methods):
        print(f"  {method[1]} → {results[i, j]:.3f} kg CO₂-eq")



acrylic acid production:
  climate change: fossil no LT → 2.289 kg CO₂-eq
  climate change: land use no LT → 0.001 kg CO₂-eq
fibre production, cotton, ginning:
  climate change: fossil no LT → 1.442 kg CO₂-eq
  climate change: land use no LT → 0.572 kg CO₂-eq
polyester fibre production, finished:
  climate change: fossil no LT → 4.488 kg CO₂-eq
  climate change: land use no LT → 0.004 kg CO₂-eq
fibre production, viscose:
  climate change: fossil no LT → 3.083 kg CO₂-eq
  climate change: land use no LT → 0.004 kg CO₂-eq
market for fibre, silk, short:
  climate change: fossil no LT → 0.010 kg CO₂-eq
  climate change: land use no LT → 0.000 kg CO₂-eq
nylon 6-6 production:
  climate change: fossil no LT → 7.096 kg CO₂-eq
  climate change: land use no LT → 0.000 kg CO₂-eq
sheep production, for wool:
  climate change: fossil no LT → 11.373 kg CO₂-eq
  climate change: land use no LT → 4.756 kg CO₂-eq
